## The setup of Project

In [ ]:
import sys
import os
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [ ]:
!pip install torch==1.7.1
pip install complexPyTorch

## Data Input

In [ ]:
#choose radar data formats
input = 'range-time' # or 'range-doppler', 'spectrograms'

In [ ]:
"""Load Data"""
if input == 'rang-time':
  data_path = 'rt_complex2.npy' # just add your path of range-time dataset (rt_complex2.npy)
if input == 'range-doppler':
  data_path = 'rd_complex2.npy' # just add your path of range-doppler dataset (rd_complex2.npy) 
if input == 'spectrograms':
  data_path = 'spec_complex2.npy' # just add your path of spectrograms dataset (spec_complex2.npy)
arr_df = np.load(data_path)
X = arr_df[:,:-1]
Y_onehot = arr_df[:,-1] - 1
arr_X = X.reshape(X.shape[0],-1,240)
arr_X = arr_X.swapaxes(1,2)
print(f"input is {input} 15-people data")

In [ ]:
import random
train_index = random.sample(range(arr_X.shape[0]), int(arr_X.shape[0]*0.8))
test_index = [i for i in range(arr_X.shape[0]) if i not in train_index]

## Complex Blocks and Models

In [ ]:
## Complex Blocks
class ComplexConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=(1,1), padding=1, dilation=1, groups=1, bias=True):
        super(ComplexConv,self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.padding = padding

        ## Model components
        self.conv_re = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.conv_im = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        
    def forward(self, x): # shpae of x : [batch,2,channel,axis1,axis2]
        real = self.conv_re(x[:,0]) - self.conv_im(x[:,1])
        imaginary = self.conv_re(x[:,1]) + self.conv_im(x[:,0])
        output = torch.stack((real,imaginary),dim=1)
        return output

In [ ]:
class CReLU(nn.Module):
    def __init__(self):
        super(CReLU,self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        ## Model components
        self.relu_re = nn.ReLU()
        self.relu_im = nn.ReLU()

    def forward(self, x): # shpae of x : [batch,2,channel,axis1,axis2]
        real = self.relu_re(x[:,0])
        imaginary = self.relu_im(x[:,1])
        output = torch.stack((real,imaginary),dim=1)
        return output

In [ ]:
class ComplexPool(nn.Module):
    def __init__(self, kernel_size, stride=(1,1), padding=0, dilation=1):
        super(ComplexPool,self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.padding = padding
        self.pool_re = nn.MaxPool2d(kernel_size,stride, padding=padding)
        self.pool_im = nn.MaxPool2d(kernel_size,stride,  padding=padding)
        
    def forward(self, x): # shpae of x : [batch,2,channel,axis1,axis2]
        real = self.pool_re(x[:,0]) 
        imaginary =self.pool_im(x[:,1]) 
        output = torch.stack((real,imaginary),dim=1)
        return output

In [ ]:
class ComplexAdaptiveAvgPool2d(nn.Module):
    def __init__(self, output_size = (1,1)):
        super(ComplexAdaptiveAvgPool2d,self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.pool_re = nn.AdaptiveAvgPool2d(output_size)
        self.pool_im = nn.AdaptiveAvgPool2d(output_size)
        
    def forward(self, x): # shpae of x : [batch,2,channel,axis1,axis2]
        real = self.pool_re(x[:,0]) 
        imaginary = self.pool_im(x[:,1]) 
        output = torch.stack((real,imaginary),dim=1)
        return output

In [ ]:
from complexPyTorch.complexLayers import ComplexBatchNorm2d
class ComplexBN(nn.Module):
    def __init__(self, num_features):
        super(ComplexBN,self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.bn = ComplexBatchNorm2d(num_features)
        
    def forward(self, x): # shpae of x : [batch,2,channel,axis1,axis2]
        data_x = np.zeros([x.shape[0],x.shape[2],x.shape[3],x.shape[4]])
        data_x = x[:,0,:,:,:].type(torch.complex64) + 1j*x[:,1,:,:,:].type(torch.complex64)
        x = self.bn(data_x)
        real = x.real
        imaginary = x.imag
        output = torch.stack((real,imaginary),dim=1)
        return output

In [ ]:
class shallow_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size=(4,4), stride = (2,2), padding=1)
        self.bn_1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size= (2, 2), stride = (2,2))
        self.relu = nn.ReLU()

        self.fc1 = nn.Linear(in_features = 64*60*30, out_features = 2048)
        self.fc2 = nn.Linear(2048, 9)

    def forward(self, x):
        x = self.pool1(self.relu(self.bn_1(self.conv1(x)))) #before flatten 
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
class shallow_ch2_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 2, out_channels = 64, kernel_size=(4,4), stride = (2,2), padding=1)
        self.bn_1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size= (2, 2), stride = (2,2))
        self.relu = nn.ReLU()

        self.fc1 = nn.Linear(in_features = 64*60*30, out_features = 2048)
        self.fc2 = nn.Linear(2048, 9)

    def forward(self, x):
        x = self.pool1(self.relu(self.bn_1(self.conv1(x)))) #before flatten 
        #print(x.shape)

        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
class deep_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size=(4,4), stride = (2,2), padding=1)
        self.bn_1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size= (2, 2), stride = (2,2))
        self.relu = nn.ReLU()
        
        self.conv2 = nn.Conv2d(64, 64, (3,3),(1,1), padding=1)
        self.bn_2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size= (2, 2), stride = (2,2))
        
        self.conv3 = nn.Conv2d(64, 128, (3,3),(1,1), padding=1)
        self.bn_3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size= (2, 2), stride = (2,2))
        
        self.conv4 = nn.Conv2d(128, 128, (2,2),(1,1), padding=1)
        self.bn_4 = nn.BatchNorm2d(128)
        self.pool4 = nn.MaxPool2d(kernel_size= (2, 2), stride = (2,1))
        
        self.conv5 = nn.Conv2d(128, 128, (2,2),(1,1), padding=1)
        self.bn_5 = nn.BatchNorm2d(128)
        self.pool5 = nn.MaxPool2d(kernel_size= (2, 2), stride = (1,1))      
        '''
        self.conv6 = nn.Conv2d(128, 128, (2,2),(1,1), padding=1)
        self.bn_6 = nn.BatchNorm2d(128)
        self.pool6 = nn.MaxPool2d(kernel_size= (2, 2), stride = (1,1))
        
        self.conv7 = nn.Conv2d(128, 128, (2,2),(1,1), padding=1)
        self.bn_7 = nn.BatchNorm2d(128)
        self.pool7 = nn.MaxPool2d(kernel_size= (2, 2), stride = (1,1))
        '''
        self.fc1 = nn.Linear(in_features = 128*8*7, out_features = 2048)
        self.fc2 = nn.Linear(2048, 9)

    def forward(self, x):
        x = self.pool1(self.relu(self.bn_1(self.conv1(x)))) #before flatten 
        x = self.pool2(self.relu(self.bn_2(self.conv2(x)))) 
        x = self.pool3(self.relu(self.bn_3(self.conv3(x)))) 
        x = self.pool4(self.relu(self.bn_4(self.conv4(x))))
        x = self.pool5(self.relu(self.bn_5(self.conv5(x)))) 
        #x = self.pool6(self.relu(self.bn_6(self.conv6(x)))) 
        #x = self.pool7(self.relu(self.bn_6(self.conv6(x)))) 

        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
class deep_ch2_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 2, out_channels = 64, kernel_size=(4,4), stride = (2,2), padding=1)
        self.bn_1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size= (2, 2), stride = (2,2))
        self.relu = nn.ReLU()
          
        self.conv2 = nn.Conv2d(64, 64, (3,3),(1,1), padding=1)
        self.bn_2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size= (2, 2), stride = (2,2))
    
        self.conv3 = nn.Conv2d(64, 128, (3,3),(1,1), padding=1)
        self.bn_3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size= (2, 2), stride = (2,2))
   
        self.conv4 = nn.Conv2d(128, 128, (2,2),(1,1), padding=1)
        self.bn_4 = nn.BatchNorm2d(128)
        self.pool4 = nn.MaxPool2d(kernel_size= (2, 2), stride = (2,1))
        
        self.conv5 = nn.Conv2d(128, 128, (2,2),(1,1), padding=1)
        self.bn_5 = nn.BatchNorm2d(128)
        self.pool5 = nn.MaxPool2d(kernel_size= (2, 2), stride = (1,1))
        
        '''
        self.conv6 = nn.Conv2d(128, 128, (2,2),(1,1), padding=1)
        self.bn_6 = nn.BatchNorm2d(128)
        self.pool6 = nn.MaxPool2d(kernel_size= (2, 2), stride = (1,1))
        '''

        self.fc1 = nn.Linear(in_features = 128*8*7, out_features = 2048)
        self.fc2 = nn.Linear(2048, 9)

    def forward(self, x):
        x = self.pool1(self.relu(self.bn_1(self.conv1(x)))) #before flatten 
        x = self.pool2(self.relu(self.bn_2(self.conv2(x)))) 
        x = self.pool3(self.relu(self.bn_3(self.conv3(x)))) 
        x = self.pool4(self.relu(self.bn_4(self.conv4(x)))) 
        x = self.pool5(self.relu(self.bn_5(self.conv5(x)))) 
        #x = self.pool6(self.relu(self.bn_6(self.conv6(x)))) 

        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
class shallow_Complex_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = ComplexConv(in_channels = 1, out_channels = 64, kernel_size=(4,4), stride = (2,2))
        self.bn_1 = ComplexBN(64)
        self.pool1 = ComplexPool(kernel_size= (2, 2), stride = (2,2))
        self.relu = CReLU()
        
        self.fc1 = nn.Linear(in_features = 2*64*60*30, out_features = 2048)
        self.fc2 = nn.Linear(2048, 9)

    def forward(self, x):
        x = self.pool1(self.relu(self.bn_1(self.conv1(x)))) #before flatten 2, 64, 60, 30  
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
class deep_Complex_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = ComplexConv(in_channels = 1, out_channels = 64, kernel_size=(4,4), stride = (2,2))
        self.bn_1 = ComplexBN(64)
        self.pool1 = ComplexPool(kernel_size= (2, 2), stride = (2,2))
        self.relu = CReLU()
        
        self.conv2 = ComplexConv(64, 64, (3,3),(1,1))
        self.bn_2 = ComplexBN(64)
        self.pool2 = ComplexPool(kernel_size= (2, 2), stride = (2,2))
        
        self.conv3 = ComplexConv(64, 128, (3,3),(1,1))
        self.bn_3 = ComplexBN(128)
        self.pool3 = ComplexPool(kernel_size= (2, 2), stride = (2,2))
        
        self.conv4 = ComplexConv(128, 128, (2,2),(1,1))
        self.bn_4 = ComplexBN(128)
        self.pool4 = ComplexPool(kernel_size= (2, 2), stride = (2,1))
        
        self.conv5 = ComplexConv(128, 128, (2,2),(1,1))
        self.bn_5 = ComplexBN(128)
        self.pool5 = ComplexPool(kernel_size= (2, 2), stride = (1,1))
        '''
        self.conv6 = ComplexConv(128, 128, (2,2),(1,1))
        self.bn_6 = ComplexBN(128)
        self.pool6 = ComplexPool(kernel_size= (2, 2), stride = (1,1))
        
        self.conv7 = ComplexConv(128, 128, (2,2),(1,1))
        self.bn_7 = ComplexBN(128)
        self.pool7 = ComplexPool(kernel_size= (2, 2), stride = (1,1))
        '''
        self.fc1 = nn.Linear(in_features = 2*128*8*7, out_features = 2048)
        self.fc2 = nn.Linear(2048, 9)

    def forward(self, x):
        x = self.pool1(self.relu(self.bn_1(self.conv1(x)))) #before flatten 
        x = self.pool2(self.relu(self.bn_2(self.conv2(x)))) 
        x = self.pool3(self.relu(self.bn_3(self.conv3(x)))) 
        x = self.pool4(self.relu(self.bn_4(self.conv4(x)))) 
        x = self.pool5(self.relu(self.bn_5(self.conv5(x)))) 
        #x = self.pool6(self.relu(self.bn_6(self.conv6(x))))
        #x = self.pool7(self.relu(self.bn_6(self.conv6(x)))) 
        #print(x.shape)
  
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

## ResNet

In [ ]:
class ResNet_complex(nn.Module):
  def __init__(
        self, num_classes: int = 9
        ):
        super(ResNet_complex, self).__init__()
        self.inplanes = 64
        self.conv1 = ComplexConv(in_channels = 1, out_channels = 64, kernel_size=(7,7), stride = (2,2), padding=3)
        self.bn1 = ComplexBN(64)
        self.relu = CReLU()
        self.pool1 = ComplexPool(kernel_size= (3, 3), stride = (2,2), padding=1)

        self.res11 = nn.Sequential(*self.make_res_block(64, 64, stride=(1,1)))
        self.res12 = nn.Sequential(*self.make_res_block(64, 64, stride=(1,1)))

        self.res21 = nn.Sequential(*self.make_res_block(64, 128, stride=(2,2)))
        self.res22 = nn.Sequential(*self.make_res_block(128, 128, stride=(1,1)))
        self.downsample2 = ComplexConv(64, 128, (1, 1), stride = (2,2), padding=0)

        self.res31 = nn.Sequential(*self.make_res_block(128, 256, stride=(2,2)))
        self.res32 = nn.Sequential(*self.make_res_block(256, 256, stride=(1,1)))
        self.downsample3 = ComplexConv(128, 256, (1, 1), stride = (2,2), padding=0)

        self.res41 = nn.Sequential(*self.make_res_block(256, 512, stride=(2,2)))
        self.res42 = nn.Sequential(*self.make_res_block(512, 512, stride=(1,1)))
        self.downsample4 = ComplexConv(256, 512, (1, 1), stride = (2,2), padding=0)

        self.avgpool = ComplexAdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512*2, num_classes) #
        
  def make_res_block(self, in_channel, out_channel, stride):  
      res_block = []
      res_block.append(ComplexConv(in_channel, out_channel, kernel_size=(3,3), stride=stride, padding=1, groups=1, bias=False, dilation=1))
      res_block.append(ComplexBN(out_channel))
      res_block.append(CReLU())
      res_block.append(ComplexConv(out_channel, out_channel, kernel_size=(3,3), padding=1, groups=1, bias=False, dilation=1))
      res_block.append(ComplexBN(out_channel))
      return res_block
      
  def forward(self, x):
     x = self.pool1(self.relu(self.bn1(self.conv1(x))))
     # building block 1
     x = x + self.res11(x)
     x = self.relu(x)
     x = x + self.res12(x)
     x = self.relu(x)
     # building block 2
     x = self.downsample2(x) + self.res21(x)
     x = self.relu(x)
     x = x + self.res22(x)
     x = self.relu(x)
     # building block 3
     x = self.downsample3(x) + self.res31(x)
     x = self.relu(x)
     x = x + self.res32(x)
     x = self.relu(x)
     # building block 4
     x = self.downsample4(x) + self.res41(x)
     x = self.relu(x)
     x = x + self.res42(x)
     x = self.relu(x)
     
     x = self.avgpool(x)
     #print(x.shape)
     x = torch.flatten(x, 1)
     x = self.fc(x)
     return x

In [ ]:
class ResNet18(nn.Module):
  def __init__(
        self,
        in_channels = 1,
        num_classes: int = 9
        ):
        super(ResNet18, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = 64, kernel_size=(7,7), stride = (2,2), padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size= (3, 3), stride = (2,2), padding=1)

        self.res11 = nn.Sequential(*self.make_res_block(64, 64, stride=(1,1)))
        self.res12 = nn.Sequential(*self.make_res_block(64, 64, stride=(1,1)))

        self.res21 = nn.Sequential(*self.make_res_block(64, 128, stride=(2,2)))
        self.res22 = nn.Sequential(*self.make_res_block(128, 128, stride=(1,1)))
        self.downsample2 = nn.Conv2d(64, 128, (1, 1), stride = (2,2))

        self.res31 = nn.Sequential(*self.make_res_block(128, 256, stride=(2,2)))
        self.res32 = nn.Sequential(*self.make_res_block(256, 256, stride=(1,1)))
        self.downsample3 = nn.Conv2d(128, 256, (1, 1), stride = (2,2))

        self.res41 = nn.Sequential(*self.make_res_block(256, 512, stride=(2,2)))
        self.res42 = nn.Sequential(*self.make_res_block(512, 512, stride=(1,1)))
        self.downsample4 = nn.Conv2d(256, 512, (1, 1), stride = (2,2))

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes) #

  def make_res_block(self, in_channel, out_channel, stride):
        res_block = []
        res_block.append(nn.Conv2d(in_channel, out_channel, kernel_size=(3,3), stride=stride, padding=1, groups=1, bias=False, dilation=1))
        res_block.append(nn.BatchNorm2d(out_channel))
        res_block.append(nn.ReLU(inplace=True))
        res_block.append(nn.Conv2d(out_channel, out_channel, kernel_size=(3,3), padding=1, groups=1, bias=False, dilation=1))
        res_block.append(nn.BatchNorm2d(out_channel))
        return res_block
    
  def forward(self, x):
     x = self.pool1(self.relu(self.bn1(self.conv1(x))))
     # building block 1
     x = x + self.res11(x)
     x = self.relu(x)
     x = x + self.res12(x)
     x = self.relu(x)
     # building block 2
     x = self.downsample2(x) + self.res21(x)
     x = self.relu(x)
     x = x + self.res22(x)
     x = self.relu(x)
     # building block 3
     x = self.downsample3(x) + self.res31(x)
     x = self.relu(x)
     x = x + self.res32(x)
     x = self.relu(x)
     # building block 4
     x = self.downsample4(x) + self.res41(x)
     x = self.relu(x)
     x = x + self.res42(x)
     x = self.relu(x)
     
     x = self.avgpool(x)
     #print(x.shape)
     x = torch.flatten(x, 1)
     x = self.fc(x)
     return x

## programme

In [ ]:
def train(dataloader, model, loss_fn, optimizer, epoch, final_epoch):
    size = len(dataloader.dataset)
    train_acc = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        train_acc += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch == final_epoch:
      print(f"epoch: {epoch}--Train accuracy is: {(100*train_acc/size):>0.1f}%")
    return train_acc/size

In [ ]:
def test(dataloader, model, loss_fn, epoch, final_epoch):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    if epoch == final_epoch:
      test_loss /= num_batches
      correct /= size
      print(f"Test Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    return correct

In [ ]:
loss_fn = nn.CrossEntropyLoss()
def adjust_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
for i in range(4):
  if i == 0:
    # multi-channel (abs): num*1*240*120
    data_x = np.zeros([arr_X.shape[0],1,arr_X.shape[1],int(arr_X.shape[2]/2)])
    data_x[:,0,:,:] =  abs(arr_X[:,:,:int(arr_X.shape[2]/2)] + 1j*arr_X[:,:,int(arr_X.shape[2]/2):])
    print("----Tradtional CNN: abs only (real number)-----")
  if i == 1:
    # multi-channel (abs&phase): num*2*240*120
    data_x = np.zeros([arr_X.shape[0],2,arr_X.shape[1],int(arr_X.shape[2]/2)])
    data_x[:,0,:,:] =  np.abs(arr_X[:,:,:int(arr_X.shape[2]/2)] + 1j*arr_X[:,:,int(arr_X.shape[2]/2):]) 
    data_x[:,1,:,:] =  np.angle(arr_X[:,:,:int(arr_X.shape[2]/2)] + 1j*arr_X[:,:,int(arr_X.shape[2]/2):]) 
    print("----CVNN Multichannel: Abs and phase----")
  if i == 2:
    # multi-channel (real & imag): num*2*240*120
    data_x = np.zeros([arr_X.shape[0],2,arr_X.shape[1],int(arr_X.shape[2]/2)])
    data_x[:,0,:,:] =  arr_X[:,:,:int(arr_X.shape[2]/2)] 
    data_x[:,1,:,:] =  arr_X[:,:,int(arr_X.shape[2]/2):]
    print("----CVNN Multichannel: Real and imaginary----")
  if i == 3:
    # DCN
    data_x = np.zeros([arr_X.shape[0],2,1, arr_X.shape[1], int(arr_X.shape[2]/2)])
    data_x[:,0,:,:,:] = arr_X[:,:,:int(arr_X.shape[2]/2)].reshape(data_x[:, 0,...].shape) #X_real
    data_x[:,1,:,:,:] = arr_X[:,:,int(arr_X.shape[2]/2):].reshape(data_x[:, 0,...].shape) #X_imag
    print("----CVNN DCN (Deep complex networks)----")
  Xtrain = data_x[train_index,...]
  Ytrain = Y_onehot[train_index,...]
  Xtest = data_x[test_index,...]
  Ytest = Y_onehot[test_index,...]
  data_train = torch.utils.data.TensorDataset(torch.from_numpy(Xtrain).type(torch.FloatTensor), torch.from_numpy (Ytrain).type(torch.LongTensor))
  data_test = torch.utils.data.TensorDataset(torch.from_numpy(Xtest).type(torch.FloatTensor), torch.from_numpy (Ytest).type(torch.LongTensor))
  batch_size = 32
  # Create data loaders.
  train_dataloader = DataLoader(data_train, batch_size=batch_size)
  test_dataloader = DataLoader(data_test, batch_size=batch_size)
  for k in range(3):
    if k == 0:
      print(f"model is plain shallow CNN (one Building Layer)")
      if i == 0:
        model = shallow_Net().to(device)
      else:
        if i == 3:
          model = shallow_Complex_Net().to(device)
        else:
          model = shallow_ch2_Net().to(device)
    if k == 1:
      print(f"model is plain deep CNN (five Building Layers)")
      if i == 0:
        model = deep_Net().to(device)
      else:
        if i == 3:
          model = deep_Complex_Net().to(device)
        else:
          model = deep_ch2_Net().to(device)
    if k == 2:
      print(f"model is ResNet18")
      if i == 0:
        model = ResNet18(in_channels = 1).to(device)
      else:
        if i == 3:
          model = ResNet_complex().to(device)
        else:
          model = ResNet18(in_channels = 2).to(device)
    epochs = 45
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
    for t in range(epochs):
      lr = 1e-4
      if t > 30:
        if t < 40:
          lr = 1e-5
        else:
          lr = 1e-6*5
      adjust_learning_rate(optimizer, lr)
      train(train_dataloader, model, loss_fn, optimizer, t, epochs-1)
      test(test_dataloader, model, loss_fn, t, epochs-1)
    print("Done!")